<a href="https://colab.research.google.com/github/Isotr0py/Acolite-GEE-Colab/blob/main/Acolite-GEE-Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Initialize Runtime
#@markdown Mount Google Drive
Mount_Drive=True #@param {type:"boolean"}
if Mount_Drive:
  from google.colab import drive
  drive.mount('/content/gdrive')
# acolite requirements
!pip install netcdf4

In [ ]:
# @title Authenticate and Initialize GEE
import ee
import geemap

#@markdown Your GEE project to initialize
project = "ee-acolite" # @param {type:"string"}
ee.Authenticate()
ee.Initialize(project=project)
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com',project='ee-entr0py')
Map=geemap.Map()

In [ ]:
# @title Clone acolite repository
# @markdown Download [Acolite](https://github.com/acolite/acolite) from Github
# Acolite can't be installed as library. It's hard coded.
%cd /content/
Release = "20231023.0" #@param ["20231023.0"]
# @markdown **<font color=red>Warning: use_latest_repo will download `main` branch instead of Release version (unstable)!!**
use_latest_repo = False #@param{type:"boolean"}
if use_latest_repo:
  !git clone --depth 1 https://github.com/acolite/acolite
else:
  !git clone --depth 1 https://github.com/acolite/acolite --branch $Release
%cd acolite
# !git clone https://github.com/acolite/acolite_luts data/LUT

In [ ]:
import ee
import geemap

#@title Setting ROI and Run acolite
# @markdown **<font color=cyan>:)  Readme**

# @markdown - **<font color=cyan>Setting the parameters**

# @markdown - **<font color=cyan>First Run will display a GEE map**

# @markdown - **<font color=cyan>Draw your ROI on map**

# @markdown - **<font color=cyan>Re-run the cell to start the process!**



# @markdown Basic settings
isodate_start='2016-01-05' #@param {type:'date'}
isodate_end='2016-01-07' #@param {type:'date'}
sensors="L8_OLI"  #@param ["L8_OLI"]
output_dir = "/content/example" # @param {type:"string"}

settings = {}
gee_settings = {}

gee_settings["output"] = output_dir
gee_settings["sensors"] = sensors
gee_settings["isodate_start"] = isodate_start
gee_settings["isodate_end"] = isodate_end
gee_settings["strict_subset"] = True
gee_settings["run_hybrid_dsf"] = False
gee_settings["run_offline_dsf"] = True

#@markdown L2W parameters
settings["l2w_parameters"]= []
Rrs = True #@param {type:"boolean"}
if Rrs:
  settings["l2w_parameters"].append('Rrs_*')

rhorc = True #@param {type:"boolean"}
if rhorc:
  settings["l2w_parameters"].append('rhorc_*')

custom_l2w_parameters = ""  #@param {type:"string"}
if custom_l2w_parameters:
  settings["l2w_parameters"] += custom_l2w_parameters.split(",")

# GEEMAP
image = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate(isodate_start,isodate_end);

#Visuaization

visualization = {
  'bands': ['B4','B3','B2'],
  'min': 0,
  'max': 0.5,
  'gamma':1.6,
}

Map.addLayer(image, visualization);
display(Map)
if Map.draw_features:
  feature = ee.FeatureCollection(Map.draw_features)
  coordinates = feature.geometry().bounds().coordinates().getInfo()[0]
  lon = [x[0] for x in coordinates]
  lat = [x[1] for x in coordinates]
  S,W,N,E = min(lat),min(lon),max(lat),max(lon)
  gee_settings["limit"] = [S,W,N,E]

  import acolite.gee
  acolite.gee.agh_run(settings=gee_settings, acolite_settings=settings)

  # @markdown **<font color=green>Use proposed SVR Zsd model**
  Zsd_SVR_plugin = True #@param{type:"boolean"}
  if Zsd_SVR_plugin:
    from glob import glob
    from pathlib import Path

    import joblib
    import numpy as np
    import xarray as xr
    import matplotlib.pyplot as plt


    l2w_files = glob(output_dir+"/*L2W_GEE_crop.nc")
    if not Path("zsd_model.pkl").exists():
      !wget https://zenodo.org/records/10665381/files/zsd_model.pkl
    model = joblib.load("zsd_model.pkl")
    for file in l2w_files:
      file = Path(file)
      parent= file.parent
      name = file.stem
      print(f"Processing ZSD: {file.name}")

      fig = plt.figure()
      ax = fig.add_subplot(111)
      plt.axis('off')

      with xr.open_mfdataset(file, parallel=True) as ds:
        x = (ds.rhorc_483-ds.rhorc_1609)/(ds.rhorc_561-ds.rhorc_1609)
        shape = x.shape
        x = x.where(~np.isnan(x), 0)
        zsd = model.predict(x.values.reshape(-1,1))
        ds_zsd = xr.Dataset(coords=ds.coords)
        ds_zsd["zsd"] = (("y","x"), zsd.reshape(shape))
        ds_zsd["zsd"] = ds_zsd.zsd.where(ds.l2_flags==0)
        ds_zsd["zsd"].attrs = {'long_name': 'Secchi Disk Depth',
                'mate': 'Zsd', 'units':"m"}
        ds_zsd["zsd"].plot.imshow(cmap="Spectral", ax=ax, vmin=int(ds_zsd["zsd"].min()), vmax=int(ds_zsd["zsd"].max()))
        ax.set_title(f"{ds.sensor}\n{ds.isodate}")
        fig.savefig(parent.joinpath(f"{name}_zsd.png"), bbox_inches="tight")
        plt.close(fig)

      ds_zsd.to_netcdf(file,mode="a")

